Evan Edelstein
EN.605.645.82.SP26

# Module 5 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

## Solving Normal Form Games

In [1]:
from typing import List, Tuple, Dict, Callable, Set

In the lecture we talked about the Prisoner's Dilemma game, shown here in Normal Form:

Player 1 / Player 2  | Defect | Cooperate
------------- | ------------- | -------------
Defect  | -5, -5 | -1, -10
Cooperate  | -10, -1 | -2, -2

where the payoff to Player 1 is the left number and the payoff to Player 2 is the right number. We can represent each payoff cell as a Tuple: `(-5, -5)`, for example. We can represent each row as a List of Tuples: `[(-5, -5), (-1, -10)]` would be the first row and the entire table as a List of Lists:

In [2]:
prisoners_dilemma = [
 [( -5, -5), (-1,-10)],
 [(-10, -1), (-2, -2)]]

prisoners_dilemma

[[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]

in which case the strategies are represented by indices into the List of Lists. For example, `(Defect, Cooperate)` for the above game becomes `prisoners_dilemma[ 0][ 1]` and returns the payoff `(-1, -10)` because 0 is the first row of the table ("Defect" for Player 1) and 1 is the 2nd column of the row ("Cooperate" for Player 2).

For this assignment, you are going write a function that uses Successive Elimination of Dominated Strategies (SEDS) to find the **pure strategy** Nash Equilibrium of a Normal Form Game. The function is called `solve_game`:

```python
def solve_game( game: List[List[Tuple]], weak=False) -> List[Tuple]:
    pass # returns strategy indices of Nash equilibrium or None.
```

and it takes two parameters: the game, in a format that we described earlier and an optional boolean flag that controls whether the algorithm considers only **strongly dominated strategies** (the default will be false) or whether it should consider **weakly dominated strategies** as well.

It should work with game matrices of any size and it will return the **strategy indices** of the Nash Equilibrium. If there is no **pure strategy** equilibrium that can be found using SEDS, return the empty List (`[]`).


<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
Do not return the payoff. That's not useful. Return the strategy indices, any other output is incorrect.
</div>

As before, you must provide your implementation in the space below, one Markdown cell for documentation and one Code cell for implementation, one function and assertations per Codecell.


---

In [3]:
def is_strongly_dominated( payoffs_c: List[int], payoffs_j: List[int]) -> bool:  # true if C is dominated by j
    return all(p1 < p2 for p1, p2 in zip(payoffs_c, payoffs_j))


In [4]:
def is_weakly_dominated( payoffs_c: List[int], payoffs_j: List[int]) -> bool:  # true if C dominated by j
    return all([p1 <= p2 for p1, p2 in zip(payoffs_c, payoffs_j)]) and any(
        [p1 < p2 for p1, p2 in zip(payoffs_c, payoffs_j)]
    )

In [5]:
assert is_weakly_dominated([0,0,0], [0,0,0]) is False

In [6]:
def is_dominated(payoffs_c: List[int], payoffs_j: List[int], weak: bool) -> bool:
    if weak:
        return is_weakly_dominated(payoffs_c, payoffs_j)
    return is_strongly_dominated(payoffs_c, payoffs_j)

In [7]:
def get_payoffs( game: List[List[Tuple[int, int]]], player: int, strategy: int, valid_rows: Tuple[int, ...], valid_cols: Tuple[int, ...],) -> List[int]:
    return [game[strategy][c][player] for c in valid_cols] if player == 0 else [game[r][strategy][player] for r in valid_rows] if player == 1 else []

In [8]:
def eliminate_dominated( game: List[List[Tuple[int, int]]], player: int, valid_rows: Tuple[int, ...], valid_cols: Tuple[int, ...], weak: bool,) -> Tuple[int, ...]:
    indices = valid_rows if player == 0 else valid_cols
    dominated_strategies = set()

    for c in indices:
        payoffs_c = get_payoffs(game, player, c, valid_rows, valid_cols)
        for j in [i for i in indices if i != c]:
            payoffs_j = get_payoffs(game, player, j, valid_rows, valid_cols)

            if is_dominated(payoffs_c, payoffs_j, weak):
                dominated_strategies.add(c)
                # break
    # take dominated_strategies and remove from
    return tuple(c for c in indices if c not in dominated_strategies)

In [9]:
def successors( game: List[List[Tuple[int, int]]], strategies: Tuple[Tuple[int, ...], Tuple[int, ...]], weak: bool,) -> List[Tuple[Tuple[int, ...], Tuple[int, ...]]]:  # return stategies to be searched after removal of dominated stategies
    valid_rows = strategies[0]
    valid_cols = strategies[1]

    successor_strategies = []

    new_rows = eliminate_dominated(game, 0, valid_rows, valid_cols, weak)
    successor_strategies.append((new_rows, valid_cols))

    new_cols = eliminate_dominated(game, 1, valid_rows, valid_cols, weak)
    successor_strategies.append((valid_rows, new_cols))
    return successor_strategies

In [10]:
def is_goal_state(strategies: Tuple[Tuple[int, ...], Tuple[int, ...]]) -> bool:
    # is there only one solution for a player left
    return len(strategies[0]) == 1 or len(strategies[1]) == 1

In [11]:
def add_strategy_to_solution(strategies: Tuple[Tuple[int, ...], Tuple[int, ...]] , solutions: Set[Tuple[int, int]]):
    for i in strategies[0]:
        for j in strategies[1]:
            solutions.add((i, j))
    return solutions

In [12]:
def add_strategies_to_frontier(next_strategies: List[Tuple[Tuple[int, ...], Tuple[int, ...]]], frontier: List[Tuple[Tuple[int, ...], Tuple[int, ...]]], explored: Set[Tuple[Tuple[int, ...], Tuple[int, ...]]]): 
    for child in next_strategies:
        if child not in explored:
            frontier.append(child)

## solve_game

(documentation)

strategies that have no eliminations should return empty list that way the search stops.
so when frontier is filled with states that have no more eliminations, they will be popped off and alg will terminate 


how do we know then that a terminal state is a valid solution - maybe there were just no eliminations? 
how could this be true: 
1) all stratagies have the same payoff - in which case we should return all of them 
2) none are dominated

goal state is one strat per player

# what happens if weakly dominated on equal payoff game (all zeros)
# weak should return false



In [ ]:
def solve_game(game: List[List[Tuple[int, int]]], weak: bool = False) -> List[Tuple[int, int]]:
    solutions: Set[Tuple[int, int]] = set()
    strategies: Tuple[Tuple[int, ...], Tuple[int, ...]] = (tuple(range(len(game))), tuple(range(len(game[0]))))
    frontier: List[Tuple[Tuple[int, ...], Tuple[int, ...]]] = [strategies]
    explored: Set[Tuple[Tuple[int, ...], Tuple[int, ...]]] = set()

    while frontier:
        strategies = frontier.pop(0)
        explored.add(strategies)

        next_strategies = successors(game, strategies, weak)

        if is_goal_state(strategies):
            solutions = add_strategy_to_solution(strategies, solutions)
            continue

        add_strategies_to_frontier(next_strategies, frontier, explored)

    return list(solutions)

In [15]:
prisoners_dilemma = [
 [( -5, -5), (-1,-10)],
 [(-10, -1), (-2, -2)]]

eql = solve_game(prisoners_dilemma, True)
print(eql)
assert eql == [(0,0)]

game2 = [
    [(10, 10), (14, 12), (14, 15)],
    [(12, 14), (20, 20), (28, 15)],
    [(15, 14), (15, 28), (25, 25)],
]
eql = solve_game(game2, True)
assert eql == [(1,1)]




[(0, 1), (1, 0), (0, 0)]


AssertionError: 

## Additional Directions

Create three games as described and according to the following:

1. Your games must be created and solved "by hand".
2. The strategy pairs must **not** be on the main diagonal (0, 0), (1, 1), or (2, 2). And the solution cannot be the same for both Game 1 and Game 2.
3. Make sure you fill out the Markdown ("?") with your game as well as the solution ("?").
4. Remember, **do not return the payoff**, return the strategy indices (a list of them).

## Before you code...

Solve the following game by hand using SEDS and weakly dominated strategies. 
The game has two (pure) Nash Equilibriums. 
You should find all of them.
This will help you think about what you need to implement to make the algorithm work.
**Hint**: You will need State Space Search from Module 1 and SEDS from Module 5 to get the full algorithm to work.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |


**Solutions**: 
- Turn 1: 
    - Player 1: compare strategies 0 (1, 3, 1), 1 (1, 3, 0) and 2 (2, 3, 0). We see strategy 1 is weekly dominated by strategy 0. 
    - remove row 1:

        | Player 1 / Player 2  | 0 | 1 | 2 |
        |----|----|----|----|
        |0  | 1/0 | 3/1 | 1/1 |
        |2  | 2/2 | 3/3 | 0/2 |

- Turn 2:
    - Player 2: compare strategies 0 (0, 2), 1 (1, 3) and 2 (1, 2). We see strategy 0 is weekly dominated by strategy 1.
    - remove col 0:
        
        | Player 1 / Player 2  | 1 | 2 |
        |----|----|----|
        |0  | 3/1 | 1/1 |
        |2  | 3/3 | 0/2 |

- Turn 3:
    - Player 1: compare strategies 0 (3, 1) and 2 (3, 0). We see strategy 0 is dominated by strategy 2.
    - remove row 0:

        | Player 1 / Player 2  | 1 | 2 |
        |----|----|----|
        |0  | 3/1 | 1/1 |
    
- Since there are no more eliminations to make the Nash Equilibriums are strategies (0,1) and (0,2).

### Test Game 1. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Strongly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ? | ? | ? |
|1  | ? | ? | ? |
|2  | ? | ? | ? |

**Solution:**? (strategy indices)

In [ ]:
test_game_1 = []

solution = solve_game(test_game_1)
print(solution)

In [ ]:
assert solution == [] # insert your solution from above.

### Test Game 2. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Weakly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ? | ? | ? |
|1  | ? | ? | ? |
|2  | ? | ? | ? |

**Solution:**? (strategy indices)

In [ ]:
test_game_2 = []

strong_solution = solve_game( test_game_2)
weak_solution = solve_game( test_game_2, weak=True)

In [ ]:
assert strong_solution == []
assert weak_solution == [] # insert your solution from above.

### Test Game 3. Create a 3x3 two player game

**that cannot be solved using the Successive Elimintation of Dominated Strategies at all**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ? | ? | ? |
|1  | ? | ? | ? |
|2  | ? | ? | ? |

**Solution:** None

In [ ]:
test_game_3 = []

strong_solution = solve_game( test_game_3)
weak_solution = solve_game( test_game_3, weak=True)

In [ ]:
assert strong_solution == []
assert weak_solution == []

### Test Game 4. Multiple Equilibria

You solved the following game by hand, above.
Now use your code to solve it.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |

**Solutions:** (copy from above)

In [ ]:
test_game_4 = [
[(1, 0), (3, 1), (1, 1)],
[(1, 1), (3, 0), (0, 1)],
[(2, 2), (3, 3), (0, 2)]]

strong_solution = solve_game( test_game_4)
weak_solution = solve_game( test_game_4, weak=True)

In [ ]:
assert strong_solution == []
assert weak_solution == [] # put solution here

## Before You Submit...

1. Did you provide output exactly as requested? **Don't forget to fill out the Markdown tables with your games**.
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.